# Crane V1 training evolution visualisation

#### Imports

In [1]:
import gym
import numpy as np

env = gym.make('crane-v1') #Load the environement

#### DQN Agent

In [2]:
import numpy as np
import random
from collections import namedtuple, deque

#from model import QNetwork # UNCOMMENT IF YOU ARE NOT IN A JUPYTER NOTEBOOK

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = 5*int(1e5)   # replay buffer size
BATCH_SIZE = 64            # minibatch size
GAMMA = 0.995 #was 0.99    # discount factor
TAU = 1e-3                 # for soft update of target parameters
LR =  5e-4                 # learning rate 
UPDATE_EVERY = 4           # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """

        
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)       

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

#### Neural Network

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64): 
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        


    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

#### Initialize agent

In [4]:
import torch
import time
from collections import deque
#from agent import Agent   # UNCOMMENT IF YOU ARE NOT IN A JUPYTER NOTEBOOK
import matplotlib.pyplot as plt
%matplotlib inline

state_size=8        #State size of  environement 
action_size=9       #Action size of enviroenemnt
seed=0
agent = Agent(state_size=8, action_size=9, seed=0) #setting the agent's parameters 

### Training Visualization

#### No training, random actions

In [5]:
#Choose model
#agent.qnetwork_local.state_dict() 

observation = env.reset()
reward_sum = 0
for t in range(200):
    env.render()
    time.sleep(0.01)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)

    reward_sum = reward_sum + reward
        
    if done:
        break
env.close()

#### 100th episode of training

Random actions but aims for x target

In [6]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_100.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final lenght is : ", state[4])
print("final angle is : ", state[2])
env.close()


Score: -4562.711673736004
final state is :  [ 1.58214525 -0.23392176  0.30147411  0.61487214 12.40948963  0.51192074
  1.          0.        ]
final x position is :  1.5821452525114708
final lenght is :  12.409489630799705
final angle is :  0.3014741071583539


#### 200th episode of training

Starts to understand how to reduce sway by raisong rope lenght

In [7]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_200.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final lenght is : ", state[4])
print("final angle is : ", state[2])
env.close()


Score: 9507.339465487948
final state is :  [0.91204795 0.07985755 0.16233935 0.1720494  9.40434934 0.16143115
 1.         0.        ]
final x position is :  0.9120479481717717
final lenght is :  9.404349336894029
final angle is :  0.16233935368246677


#### 300th episode of training

Understand that going slow might be good for better controlling

In [8]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_300.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final lenght is : ", state[4])
print("final angle is : ", state[2])
env.close()


Score: 7392.3348890094885
final state is :  [ 0.18909118  0.12135814  0.08886969 -0.77876979  2.29451699 -2.38232993
  1.          0.        ]
final x position is :  0.18909118290167773
final lenght is :  2.29451698915525
final angle is :  0.08886969267796581


#### 700th episode of training

Starts realising that sway angle AND rope lenght is important to get maximum rewards

In [10]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_700.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final lenght is : ", state[4])
print("final angle is : ", state[2])
env.close()


Score: 12098.141021047952
final state is :  [ 0.81195412  0.05868317 -0.1645917   0.13838842  1.27192223  0.11467967
  1.          0.        ]
final x position is :  0.8119541208840354
final lenght is :  1.2719222266975019
final angle is :  -0.16459169803812634


#### 800th episode of training

Finds a path where it raises the rope to stabilyse the mass and then aims for the target

Best model for this training

Can not stabilyse perfectly arround the target

In [17]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_800.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final lenght is : ", state[4])
print("final angle is : ", state[2])
env.close()


Score: 154500.34814586674
final state is :  [ 1.19053576  0.03917693  0.17225983 -0.35216672  2.31128437  0.32887241
  1.          0.        ]
final x position is :  1.1905357597484272
final lenght is :  2.3112843735027626
final angle is :  0.17225982592303204


#### 1300th episode of training

Completly diverges because it did not solve the environement

In [16]:
#Choose model
#agent.qnetwork_local.state_dict() 

agent.qnetwork_local.load_state_dict(torch.load('Episode_1300.pth'))

env_info = env.reset()                             # reset the environment
state = env.state                                  # get the current state
score = 0                                          # initialize the score
t = 0
max_t = 1000                                       # Number of timesteps
for t in range(max_t) :
    env.render()
    #time.sleep(0.008)
    action = agent.act(state)                      # select an action
    env_info = env.step(action)                    # send the action to the environment
    next_state = env_info[0]                       # get the next state
    reward = env_info[1]                           # get the reward
    done = env_info[2]                             # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    t += 1.0    
    if done:                                       # exit loop if episode finished
        print("Total steps : ", t)
        print("Total time is : ", env.tau * t)
        break
        
print("Score: {}".format(score))
print("final state is : ", state)
print("final x position is : ", state[0])
print("final lenght is : ", state[4])
print("final angle is : ", state[2])
env.close()


Score: -11118.580196654351
final state is :  [-9.98175399e+02 -9.99154448e+01 -1.85619002e-01  1.41678360e+00
  1.00000000e-01 -2.23175256e-01  1.00000000e+00  0.00000000e+00]
final x position is :  -998.1753992359584
final lenght is :  0.1
final angle is :  -0.18561900229183045
